In [1]:
import numpy as np
import pandas as pd
import os, sys
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')


%cd C:\emobility

C:\emobility


## Car assignment

In [2]:
df_act = pd.read_csv(f'./dbs/agents/1_activity_plans.csv')


emission_factor = pd.read_csv(f'./dbs/emission/EmissionFactor.csv', delimiter=";")
PHEV = emission_factor[(emission_factor['carType']== 'Car_PHEV_bensin') | (emission_factor['carType']== 'Car_PHEV_diesel')].mean(axis=0)
emission_factor = emission_factor.append( PHEV, ignore_index=True ).drop([4, 5], axis=0)
emission_factor.iloc[5,0] = 'Car_PHEV'
emission_factor.replace(['Car_gasoline','Car_diesel','Car_ethanol','Car_gas','Car_el ','Car_PHEV'],[0,1,4,5,2,3], inplace=True)




car_distribution = pd.read_csv(f'./dbs/emission/CarDistribution.csv', delimiter=";")
car_distribution.loc[:,['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']] = car_distribution.loc[:,['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']].div(car_distribution['Total'], axis=0)

car_distribution = car_distribution.drop(['Total'], axis=1)



In [3]:
def carType_assignment(row, car_distribution):
    if row['deso'][:2] == '14':
        prob = car_distribution[['Gasoline', 'Diesel', 'El', 'PHEV', 'Ethanol', 'Gas']][car_distribution.Kod==int(row['deso'][:4])].values.tolist()
        pred = np.random.multinomial(1, prob[0], 1)
        carType = pred[0].tolist().index(1)
    else:
        # for those not in VG, use VG general distribution
        prob = [0.579957, 0.354470, 0.003693, 0.006844, 0.043670, 0.011365]
        pred = np.random.multinomial(1, prob, 1)
        carType = pred[0].tolist().index(1)

    return carType




In [5]:
df_act_car_segments = df_act.drop_duplicates(subset=['person'])

df_act_car_segments['carType'] = 100

df_act_car_segments['carType'] = df_act_car_segments.apply( lambda row: carType_assignment(row, car_distribution), axis=1)

In [6]:
print('car distribution by fuel type at the aggregated level')
df_act_car_segments.carType.value_counts(normalize=True)

car distribution by fuel type at the aggregated level


0    0.579364
1    0.355368
4    0.043325
5    0.011199
3    0.006969
2    0.003776
Name: carType, dtype: float64

## Emission calculation
### the road network classification by the population density in DeSO where the road link passes

In [7]:
import geopandas as gpd

#A DeSO in category A is rural areas. B is mostly located in a suburban areas. C is mostly located in the municipality's central location.
# read the road network and DeSO zones

road_network = gpd.read_file(f'./dbs/emission/road/5_road_network_with_slope.shp')

road_network = road_network[['link_id','geometry']]
road_network = road_network.reset_index()

DeSO = gpd.read_file(f'./dbs/emission/deso_statistik_shp/Fordon_region.shp')
DeSO = DeSO[['Deso','geometry']]
DeSO['deso_category'] = DeSO['Deso'].str[4]


DeSO = DeSO.to_crs(road_network.crs)

In [8]:
from spatial_operations import spatial_join


if __name__ == '__main__':
    road_network = spatial_join(road_network, DeSO)

# the links crossing multiple DeSO are duplicated in the spatial join process.
# the link id is no unique, and it crosses multiple DeSO zones when it be combined

In [12]:
road_network_Noduplicate = road_network.copy()

road_network_Noduplicate['deso_category'] = road_network_Noduplicate['deso_category'].fillna('M')


In [13]:
road_network_Noduplicate = road_network_Noduplicate.groupby(['link_id']).agg({'deso_category': ' '.join}).reset_index()

In [14]:
#road_network_Noduplicate.deso_category.value_counts()

In [15]:
def deso_categories(row):
    if len(row['deso_category'])==1:
        deso_category = row['deso_category']
    elif 'M' in row['deso_category']:
        deso_category = 'M'
    else:
        deso_category = "".join(set(row['deso_category'])).replace(" ", "")
        if (len(deso_category)>1) & ("C" in deso_category) :
            deso_category = 'M'
        elif (len(deso_category)>1) :
            deso_category = 'B'
        #elif: (len(deso_category)>1) & ("A" in deso_category)  :
            #deso_category = 'B'

    return deso_category

In [17]:
road_network_Noduplicate['deso_category'] = road_network_Noduplicate.apply( lambda row: deso_categories(row), axis=1)

In [18]:

road_network_Noduplicate['deso_category'].value_counts(normalize=True)

C    0.465838
A    0.429098
B    0.094436
M    0.010628
Name: deso_category, dtype: float64

In [20]:
road_network_Deso = road_network.copy()
road_network_Deso['Deso'] = road_network_Deso['Deso'].fillna('M')
road_network_Deso = road_network_Deso.groupby('link_id')['Deso'].apply(list).reset_index()


### car movement trajectories on the road network

In [21]:
df_act_car_tra = pd.read_csv(f'./dbs/emission/Car_trajectories.csv', delimiter=",")


In [22]:
df_act_car_tra.sort_values(["person", "seq"], inplace=True)

In [23]:
df_act_car_tra = pd.merge(df_act_car_tra[['person', 'seq', 'time', 'purpose', 'type', 'link', 'distance_driven']], road_network_Noduplicate[['link_id', 'deso_category']], left_on= "link", right_on="link_id", how="left")

In [24]:
df_act_car_tra = pd.merge(df_act_car_tra, road_network_Deso, left_on= "link", right_on="link_id", how="left")

In [27]:
df_act_car_tra['distance_driven_shift'] = df_act_car_tra['distance_driven'].shift(-1)
df_act_car_tra['person_shift'] = df_act_car_tra['person'].shift(-1).fillna(0).astype('int')

df_act_car_tra['link_distance']=0

In [28]:
df_act_car_tra['link_distance'] = df_act_car_tra.apply( lambda row: row['distance_driven_shift']-row['distance_driven'] if row['person'] == row['person_shift'] else 0, axis=1)

In [29]:
# According to link distance (I have checked in Arcmap), shift rows.
df_act_car_tra['link_distance'] = df_act_car_tra['link_distance'].shift(1).fillna(0)

### activity purpose deduction

In [30]:
df_OTP = pd.read_csv(f'./dbs/agents/od_pairs_vg.csv')

df_OTP_last_activity = df_OTP.drop_duplicates(subset=['person'], keep='last')


In [32]:
df_act_last_activity = df_act_car_tra[df_act_car_tra.type == 'vehicle leaves traffic']

df_act_last_activity = df_act_last_activity.drop_duplicates(subset=['person','type'], keep='last')

In [33]:
df_act_last_activity = pd.merge(df_act_last_activity[['person','seq']], df_OTP_last_activity[["person", "act_purpose"]], left_on= "person", right_on="person", how="left")


In [34]:
df_act_car_tra = pd.merge(df_act_car_tra, df_act_last_activity, on= ["person",'seq'], how="left")

In [35]:
df_act_car_tra['purpose'] = np.where(pd.isna(df_act_car_tra.purpose), df_act_car_tra['act_purpose'], df_act_car_tra['purpose'])

In [36]:
df_act_car_tra.sort_values(["person", "seq"], inplace=True)

In [37]:
df_act_car_tra['purpose'].bfill(axis ='rows', inplace=True)


In [38]:
df_act_car_tra.drop(['person_shift', 'distance_driven_shift','act_purpose', 'link_id_x', 'link_id_y'], inplace=True, axis=1)

### assign activity IDs

In [42]:
act_id = df_act_car_tra[df_act_car_tra.type == 'vehicle leaves traffic']
act_id['act_id'] = act_id.groupby('person').cumcount()

df_act_car_tra = pd.merge(df_act_car_tra, act_id[['act_id']], left_index=True, right_index=True, how="left")



In [43]:
df_act_car_tra['act_id'].bfill(axis ='rows', inplace=True)
df_act_car_tra['act_id'] = df_act_car_tra['act_id'].astype('int')



### calculate car emission by the links

In [44]:
### add car with the fuel type to the trajectories

df_act_car_tra = pd.merge(df_act_car_tra, df_act_car_segments[["person", "carType"]],  left_on="person", right_on="person", how="left")

df_act_car_tra = pd.merge(df_act_car_tra, emission_factor, left_on="carType",
                                       right_on="carType", how="left")

In [45]:
def link_emission(row):
    if row['deso_category'] == 'C':
        link_emission = row['link_distance'] * row['TTW_city']
    elif row['deso_category'] == 'M':
        link_emission = row['link_distance'] * row['TTW_mix']
    elif (row['deso_category'] == 'A') | (row['deso_category'] == 'B'):
        link_emission = row['link_distance'] * row['TTW_rural']
    return link_emission




In [46]:
df_act_car_tra['link_emission'] = df_act_car_tra.apply(lambda row: link_emission(row), axis=1)
df_act_car_tra['link_emission'].sum()

2511757.352340936

In [55]:
df_act_car_tra.to_pickle(f"./dbs/output/emission_bylink.pkl")

In [47]:




# df_act_car_tra_bycategories = df_act_car_tra.groupby(['person', 'act_id', 'deso_category'])[
#     'link_distance'].sum().reset_index()
#
# ### add car with the fuel type to the trajectories
#
# df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, df_act_car_segments[["person", "carType"]], left_on="person", right_on="person", how="left")
# df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, emission_factor, left_on="carType", right_on="carType", how="left")

In [48]:
# df_act_car_tra_bycategories = df_act_car_tra.groupby(['person','act_id','deso_category'])['link_distance'].sum().reset_index()
#

In [49]:
# ### add car with the fuel type to the trajectories
#
# df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, df_act_car_segments[["person", "carType"]], left_on= "person", right_on="person", how="left")

In [50]:
# df_act_car_tra_bycategories = pd.merge(df_act_car_tra_bycategories, emission_factor, left_on= "carType", right_on="carType", how="left")

In [51]:
# def link_emission(row):
#     if row['deso_category']=='C':
#         link_emission = row['link_distance']*row['TTW_city']
#     elif row['deso_category']=='M':
#         link_emission = row['link_distance']*row['TTW_mix']
#     elif (row['deso_category']=='A')|(row['deso_category']=='B'):
#         link_emission = row['link_distance']*row['TTW_rural']
#     return link_emission

In [52]:
# df_act_car_tra_bycategories['link_emission'] = df_act_car_tra_bycategories.apply( lambda row: link_emission(row), axis=1)

In [53]:
# df_act_car_tra_bycategories['link_emission'].sum()

In [54]:
# df_act_car_byactivities = df_act_car_tra_bycategories.groupby(['person','act_id'])['link_emission'].sum().reset_index()
#